# Template - Strategy - Backtesting 

### Import Library

In [22]:
import numpy as np
import pandas as pd
import numpy as np
import pandas_ta as ta
from backtesting.backtesting import Backtest, Strategy
from backtesting._plotting import set_bokeh_output
set_bokeh_output(notebook=False)

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 120
import warnings
warnings.filterwarnings('ignore')

### Load Price Data

In [23]:
import os
from pathlib import Path
notebook_path = os.getcwd()
algo_dir = Path(notebook_path).parent.parent
csv_file = str(algo_dir) + '/vn-stock-data/VN30ps/VN30F1M_5minutes.csv'
is_file = os.path.isfile(csv_file)
if is_file:
    dataset = pd.read_csv(csv_file, index_col='Date', parse_dates=True)
else:
    print('remote')
    dataset = pd.read_csv("https://raw.githubusercontent.com/zuongthaotn/vn-stock-data/main/VN30ps/VN30F1M_5minutes.csv", index_col='Date', parse_dates=True)

In [24]:
data = dataset.copy()

In [25]:
# data = data[(data.index > '2020-11-01 00:00:00') & (data.index < '2024-10-01 00:00:00')]
data = data[data.index > '2020-11-01 00:00:00']

In [26]:
data

,Open,High,Low,Close,Volume
Date,,,,,
2020-11-02 09:00:00,900.1,900.2,899.3,900.1,1910
2020-11-02 09:05:00,900.2,900.2,898.7,899.4,1670
2020-11-02 09:10:00,899.5,900.0,899.0,899.5,1329
2020-11-02 09:15:00,899.4,899.5,898.2,898.6,1722
2020-11-02 09:20:00,898.5,898.6,896.5,898.2,2939
...,...,...,...,...,...
2025-02-07 14:15:00,1343.3,1344.4,1342.4,1344.2,5075
2025-02-07 14:20:00,1344.2,1344.7,1343.6,1344.0,4865
2025-02-07 14:25:00,1344.1,1344.2,1342.8,1343.3,5013


In [27]:
def prepare_data(df, lookback=50):
    df["high"] = df["High"].rolling(lookback).max()
    df["low"] = df["Low"].rolling(lookback).min()
    df["fib_38"] = df["high"] - (df["high"] - df["low"]) * 0.382
    df["fib_50"] = df["high"] - (df["high"] - df["low"]) * 0.5
    df["fib_61"] = df["high"] - (df["high"] - df["low"]) * 0.618
    return df

In [28]:
class MainStrategy(Strategy):
    risk_reward_ratio = 2
    risk_percent = 0.02

    def init(self):
        self.fib_38 = self.I(lambda: self.data.high - (self.data.high - self.data.low) * 0.382)
        self.fib_50 = self.I(lambda: self.data.high - (self.data.high - self.data.low) * 0.5)
        self.fib_61 = self.I(lambda: self.data.high - (self.data.high - self.data.low) * 0.618)

    def next(self):
        if self.position:
            return
        cash_risk = self.equity * self.risk_percent
        
        if self.data.Low[-1] <= self.fib_50[-1] and self.data.Close[-1] > self.fib_50[-1]:
            sl = self.fib_61[-1] * 0.99
            tp = self.data.Close[-1] + (self.data.Close[-1] - sl) * self.risk_reward_ratio
            risk_per_unit = max(self.data.Close[-1] - sl, 0.0001)
            position_size = max(cash_risk / risk_per_unit, 1)
            self.buy(size=1, sl=sl, tp=tp)

        elif self.data.High[-1] >= self.fib_50[-1] and self.data.Close[-1] < self.fib_50[-1]:
            sl = self.fib_38[-1] * 1.01
            tp = self.data.Close[-1] - (sl - self.data.Close[-1]) * self.risk_reward_ratio
            risk_per_unit = max(sl - self.data.Close[-1], 0.0001)
            position_size = max(cash_risk / risk_per_unit, 1)
            self.sell(size=1, sl=sl, tp=tp)

In [29]:
prepared_data = prepare_data(data)
prepared_data.dropna(inplace=True)

In [30]:
bt = Backtest(prepared_data, MainStrategy, commission=.0003, exclusive_orders=True)
stats = bt.run()

Start                     2020-11-02 14:30:00
End                       2025-02-07 14:45:00
Duration                   1558 days 00:15:00
Exposure Time [%]                   77.365351
Equity Final [$]                  9648.968972
Equity Peak [$]                  10391.753966
Return [%]                           -3.51031
Buy & Hold Return [%]               48.103638
Return (Ann.) [%]                   -0.842761
Volatility (Ann.) [%]                2.050569
Sharpe Ratio                        -0.410989
Sortino Ratio                       -0.585765
Calmar Ratio                        -0.107217
Max. Drawdown [%]                   -7.860298
Avg. Drawdown [%]                   -0.155434
Max. Drawdown Duration     1095 days 05:10:00
Avg. Drawdown Duration       10 days 18:24:00
# Trades                                  276
Win Rate [%]                        31.884058
Best Trade [%]                       7.653425
Worst Trade [%]                     -3.722697
Avg. Trade [%]                    

In [31]:
stats

Start                     2020-11-02 14:30:00
End                       2025-02-07 14:45:00
Duration                   1558 days 00:15:00
Exposure Time [%]                   77.365351
Equity Final [$]                  9648.968972
Equity Peak [$]                  10391.753966
Return [%]                           -3.51031
Buy & Hold Return [%]               48.103638
Return (Ann.) [%]                   -0.842761
Volatility (Ann.) [%]                2.050569
Sharpe Ratio                        -0.410989
Sortino Ratio                       -0.585765
Calmar Ratio                        -0.107217
Max. Drawdown [%]                   -7.860298
Avg. Drawdown [%]                   -0.155434
Max. Drawdown Duration     1095 days 05:10:00
Avg. Drawdown Duration       10 days 18:24:00
# Trades                                  276
Win Rate [%]                        31.884058
Best Trade [%]                       7.653425
Worst Trade [%]                     -3.722697
Avg. Trade [%]                    

In [32]:
stats['_trades']

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Tag,Duration
0,1,44,257,905.07144,935.000000,29.928560,0.033068,2020-11-03 13:55:00,2020-11-10 09:00:00,None,6 days 19:05:00
1,-1,304,459,927.62163,941.100000,-13.478370,-0.014530,2020-11-10 14:20:00,2020-11-16 09:00:00,None,5 days 18:40:00
2,-1,480,660,933.71980,946.300000,-12.580200,-0.013473,2020-11-16 10:45:00,2020-11-19 14:25:00,None,3 days 03:40:00
3,1,692,861,945.78365,969.049140,23.265490,0.024599,2020-11-20 11:25:00,2020-11-25 14:10:00,None,5 days 02:45:00
4,1,864,1179,966.58989,990.107484,23.517594,0.024330,2020-11-25 14:25:00,2020-12-04 09:30:00,None,8 days 19:05:00
...,...,...,...,...,...,...,...,...,...,...,...
271,-1,52558,52827,1329.80094,1345.320202,-15.519262,-0.011670,2024-12-18 09:15:00,2024-12-25 10:35:00,None,7 days 01:20:00
272,1,52878,53132,1349.90485,1334.090340,-15.814510,-0.011715,2024-12-26 10:35:00,2025-01-03 11:05:00,None,8 days 00:30:00
273,-1,53195,53410,1327.20172,1298.873268,28.328452,0.021344,2025-01-06 13:30:00,2025-01-10 14:25:00,None,4 days 00:55:00
274,1,53458,53719,1297.28907,1328.698800,31.409730,0.024212,2025-01-13 14:10:00,2025-01-21 09:00:00,None,7 days 18:50:00


In [ ]:
# bt.plot()